# **Experiment B1**: proof of concept local refinement
Riemannian gradient descent with local regular integration

Vary noise level and number of images

In [1]:
# Mount Google Drive
from google.colab import drive # import drive from google colab
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
%cd /content/drive/MyDrive/PhD/Projects/2 - Cryo-EM/src/Cryo-EM

[Errno 2] No such file or directory: '/content/drive/MyDrive/PhD/Projects/2 - Cryo-EM/src/Cryo-EM'
/Users/wdiepeveen/Documents/PhD/Projects/2 - Cryo-EM/src/src/Cryo-EM/projects/rkhs_lifting/experiments/experimentB1


In [2]:
# install libraries in Colab env
!pip install mrcfile
!pip install aspire
!pip install quaternionic
!pip install pykeops[colab] > install.log

You should consider upgrading via the '/Users/wdiepeveen/PycharmProjects/Cryo-EM/bin/python -m pip install --upgrade pip' command.


You should consider upgrading via the '/Users/wdiepeveen/PycharmProjects/Cryo-EM/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/wdiepeveen/PycharmProjects/Cryo-EM/bin/python -m pip install --upgrade pip' command.
zsh:1: no matches found: pykeops[colab]


In [ ]:
%cd ../../../..

In [1]:
%cd /Users/wdiepeveen/Documents/PhD/Projects/2 - Cryo-EM/src/src/Cryo-EM

/Users/wdiepeveen/Documents/PhD/Projects/2 - Cryo-EM/src/src/Cryo-EM


In [2]:
import numpy as np
import os
import logging

from projects.rkhs_lifting.experiments.experimentB1.E1_run import run_experiment
from projects.rkhs_lifting.experiments.experimentB1.E2_post_processing import post_processing

from tools.exp_tools import Exp

logger = logging.getLogger(__name__)

[pyKeOps]: Warning, no cuda detected. Switching to cpu only.


In [3]:
# Set data path
data_dir = os.path.join("results","expA2_21-10-27_10-39-37_L65_N512_r2_k90_lamV10_lamD-10")
data_filename = "solver_data_2SNR_512N"
# data_path = os.path.join(data_dir, data_filename)

# Set results folder if skip_preprocessing
# results_folder = "results/lifting_21-04-21_11-09-45"  # "results/lifting_21-04-26_12-22-06"
results_folder ="results/expB1_21-11-03_16-27-07_L65_N512_k90_l10_i8_lamV10"

In [8]:
# TODO load data from experiment A: initial volume and images (gt rots)
# TODO select a subset of these images to test the algorithm over

# Experiment parameters
img_size = 65  # was 65 before and was 129 in rotation estimation paper
snr = 1 / 2  # [1/2, 1/4, 1/8, 1/16, 1/32, 1/64, 1/128, 1/256]
num_imgs = 512  # 512 capped
k = 90
l = 10
kernel_radius = np.pi/k  # radius of kernel
i = 0.8
# integrator_radius = np.sqrt(2) *2 * kernel_radius / (l-1)  # i * kernel_radius
integrator_radius = i * kernel_radius
# print("i={}".format(np.sqrt(2) *2 / (l-1)))
volume_reg_param = 1e10
rots_batch_size = 8192

gd_step_size = 8  # TODO test in run_
stop_rots_gd = 0

# Directly start experiment
exp = Exp()

exp.begin(prefix="expB1" , postfix="L{}_N{}_k{}_l{}_i{}_lamV{}_gdMaxIter{}_gdStep{}".format(img_size,num_imgs,k,l,int(10*i),int(np.log(volume_reg_param)/np.log(10)),stop_rots_gd,gd_step_size))
exp.dbglevel(4)

2021-11-04 14:33:52,255 INFO Results folder is results/expB1_21-11-04_14-33-52_L65_N512_k90_l10_i8_lamV10_gdMaxIter0_gdStep8


In [5]:
# Experiment

logger.info("Start Experiment")
logger.info("Running Experiment for SNR = {} and {} images".format(snr, num_imgs))

run_experiment(exp=exp,
               num_imgs=num_imgs,
               snr=snr,
               img_size=img_size,
               kernel_radius=kernel_radius,
               l=l,
               integrator_radius=integrator_radius,
               stop_rots_gd=stop_rots_gd,
               gd_step_size=gd_step_size,
               volume_reg_param=volume_reg_param,
               data_dir=data_dir,
               data_filename=data_filename,
               )

2021-11-04 14:32:44,133 INFO Start Experiment
2021-11-04 14:32:44,135 INFO Running Experiment for SNR = 0.5 and 512 images
2021-11-04 14:32:44,137 INFO This experiment illustrates orientation refinement using a lifting approach
2021-11-04 14:32:46,012 INFO Load 3D map and downsample 3D map to desired grids of 65 x 65 x 65.
Start solver
Start Riemannian Gradient Descent Solver
2021-11-04 14:32:46,027 INFO Compute adjoint forward mapping on the images
2021-11-04 14:32:46,028 INFO Running through projections 0/512 = 0.0%
2021-11-04 14:32:46,304 INFO Trying NFFT backend finufft
2021-11-04 14:32:46,324 INFO NFFT backend finufft usable.
2021-11-04 14:32:46,326 INFO Trying NFFT backend cufinufft
2021-11-04 14:32:46,329 INFO NFFT backend cufinufft not usable:
	No module named 'pycuda'
2021-11-04 14:32:46,330 INFO Trying NFFT backend pynfft
2021-11-04 14:32:46,331 INFO NFFT backend pynfft not usable:
	No module named 'pynfft'
2021-11-04 14:32:46,332 INFO Selected NFFT backend = finufft.
2021-11

In [7]:
logger.info("Start Postprocessing")
# Postprocessing
post_processing(exp=exp,
               num_imgs=num_imgs,
               snr=snr,
#                results_folder=results_folder #?
               )


2021-11-04 14:33:18,837 INFO Start Postprocessing
2021-11-04 14:33:18,839 INFO Postprocessing started
2021-11-04 14:33:18,864 INFO MSE deviation of the estimated initial rotations using register_rotations : 0.004892074763939951
2021-11-04 14:33:18,879 INFO MSE deviation of the estimated GD-refined rotations using register_rotations : 0.004892074298165483
2021-11-04 14:33:18,895 INFO MSE deviation of the estimated reference GT rotations using register_rotations : 3.7868465546589856e-14
